# Исследование факторов, влияющих на оценки студентов на экзаменах

Ежегодно сотрудники некоторой компании проходят аттестацию, где по 100-балльной шкале получают оценки за уровень знаний по специальности (erudition_score), за коммуникативные навыки (communication_score) и за уровень продуктивности на работе (productivity).

Каждый год сотрудники могут улучшать свои навыки, проходя подготовительный курс.

Также на результаты влияют и другие факторы (уровень образования и, как ни странно, состав обеда, оценка сотрудника начальником - от А до E, пол сотрудника).

## Цели
#### 1. Понять, как различные факторы (экономические, социальные, личные) влияют на оценки студентов на экзаменах 
#### 2. Познакомиться с библиотекой seaborn.
#### 3. Применить линейную регрессию для предсказания среднего балла за экзамены.

Что нового рассказать:

* как кодировать категориальные признаки
* по какой формуле работает линейная регрессия и как оценивать её качество

#### Импортируем необходимые библиотеки

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#### Зафиксируем минимальный балл для прохождения аттестации (экзамена)

In [ ]:
passmark = 40

#### Считаем данные из csv файла EmployeesPerformance.csv

* rating - рейтинг успеваемости (группа А - низкий рейтинг, E - высокий)
* old_mean_score - средний балл за экзамен в прошлом году (в долях 1; можно перевести в 100-балльную шкалу умножением на 100)

In [ ]:
df = pd.read_csv("EmployeesPerformance2021.csv")

#### Напечатаем первые несколько строк в таблице

In [ ]:
# your code here

#### Выведем на экран размеры полученной таблицы

In [ ]:
df.shape

In [ ]:
df.info()

#### Выведем на экран статистические характеристики числовых столбцов таблицы (минимум, максимум, среднее значение, стандартное отклонение)

In [ ]:
# your code here

Посмотрим на распределение баллов по каждому экзамену.

In [ ]:
plt.figure(figsize=(14,6))
plt.subplot(1, 4, 1)
plt.title('ERUDITION SCORES')
sns.violinplot(y='erudition_score',data=df,palette='summer',linewidth=3)
plt.subplot(1, 4, 2)
plt.title('COMMUNICATION SCORES')
sns.violinplot(y='communication_score',data=df,palette='Wistia_r',linewidth=3)
plt.subplot(1, 4, 3)
plt.title('PRODUCTIVITY')
sns.violinplot(y='productivity',data=df,palette='spring',linewidth=3)
plt.show()

Из графиков видно, что большинство сотрудников набрали 60-80 по тесту на эрудицию и 50-80 по тестам на коммуникабельность и продуктивность.

#### Проверьте, есть ли в таблице пропущенные значения

In [ ]:
df.isnull().sum()

**Посмотрим, как зависят оценки от того, проходил ли сотрудник курс для подготовки к сдаче экзамена (test preparation course)
и от пола (gender)**

Постройте график (гистограмму, любую нравящуюся вам визуализацию) зависимости оценок за эрудицию (erudition score) от test preparation course и от gender. То же самое - для оценок за communication и productivity.

In [ ]:
plt.figure(figsize=(20,8))
plt.subplot(1, 3, 1)
sns.barplot(x='test preparation course',y='erudition_score',data=df,hue='gender',\
           palette='summer')
plt.title('ERUDITION SCORES')
plt.subplot(1, 3, 2)
sns.barplot(x='test preparation course',y='communication_score',data=df,hue='gender',\
           palette='summer')
plt.title('COMMUNICATION SCORES')
plt.subplot(1, 3, 3)
sns.barplot(x='test preparation course',y='productivity',data=df,hue='gender',\
           palette='summer')
plt.title('PRODUCTIVITY')
plt.show()

Какие выводы можно сделать из этих графиков?

Можно визуализировать все попарные зависимости между числовыми признаками.

In [ ]:
sns.pairplot(data=df,hue='gender',plot_kws={'alpha':0.2},palette='hot')

Как влияет lunch на оценки?

Выведем на экран все различные значения из столбца lunch

In [ ]:
# your code here

Построим barplot, отражающий зависимость оценок за erudition от test preparation course; сделаем разделение по 'lunch'.
То же самое - для communication и productivity.

In [ ]:
#your code here
plt.figure(figsize=(20,8))
plt.subplot(1, 3, 1)
sns.barplot(x='lunch',y='erudition_score',data=df,hue='gender',\
           palette='winter')
plt.title('ERUDITION SCORES')
plt.subplot(1, 3, 2)
sns.barplot(x='lunch',y='communication_score',data=df,hue='gender',\
           palette='winter')
plt.title('COMMUNICATION SCORES')
plt.subplot(1, 3, 3)
sns.barplot(x='lunch',y='productivity',data=df,hue='gender',\
           palette='winter')
plt.title('PRODUCTIVITY')
plt.show()

Как зависят баллы от типа lunch?

####  Исследуем оценки за эрудицию (erudition)

Построим гистограмму (sns.countplot) распределения оценок за эрудицию.

In [ ]:
plt.figure(figsize=(5,5))
sns.countplot(x='erudition_score',data=df,palette='summer')
plt.show()

#### Сколько сотрудников успешно сдали тест на эрудицию?

Создайте новый столбец в таблице df под названием Erudition_PassStatus и запишите в него F, если сотрудник не сдал тест на эрудицию (балл за экзамен < passmark), и P иначе.

Посчитайте количество сотрудников, сдавших и не сдавших экзамен по математике.

In [ ]:
#your code here
df['Erudition_PassStatus'] = np.where(df['erudition_score'] < passmark, 'F', 'P')

#or

df['Erudition_PassStatus'] = df['erudition_score'].apply(lambda x: 'F' if x < passmark \
                                              else 'P')

In [ ]:
df.Erudition_PassStatus

Как зависит успех сдачи экзамена от уровня образования? 

Постройте гистограмму (countplot) распределения сотрудников по уровням образования

In [ ]:
fig,ax=plt.subplots()
sns.countplot(x='level of education',data=df,hue='Erudition_PassStatus')
fig.autofmt_xdate()

#### Сколько сотрудников успешно сдали все экзамены?

Создадим столбец OverAll_PassStatus и запишем в него для каждого сотрудника 'F', если он не сдал хотя бы один из трех экзаменов, а иначе 'P'.

Посчитаем количество сотрудников, которые сдали все экзамены.

In [ ]:
df['OverAll_PassStatus'] = df.apply(lambda x: 'P' if x['erudition_score'] >= passmark and \
                                                     x['communication_score'] >= passmark and \
                                                     x['productivity'] >= passmark \
                                         else 'F', axis=1)
df.head()

Создадим диаграмму, отображающую зависимость OverAll_PassStatus от level of education

In [ ]:
fig,ax=plt.subplots()
sns.countplot(x='level of education',hue='OverAll_PassStatus',data=df)
fig.autofmt_xdate()

#### Найдем распределение средней оценки за все экзамены

Создадим столбец Percentage, в который запишем средний балл сотрудника за все дисциплины.

Нарисуем гистограмму распределения среднего балла.

In [ ]:
df['Percentage'] = (df['erudition_score'] + df['communication_score'] + df['productivity']) / 3
df.head()

**Средний балл в зависимости от уровня образования**

In [ ]:
fig,ax=plt.subplots()
sns.barplot(x='level of education',y='Percentage',data=df,palette='Wistia')
fig.autofmt_xdate()

Посмотрим на лучших сотрудников

In [ ]:
df[(df['erudition_score'] > 90) & (df['communication_score'] > 90) & (df['productivity']>90)]\
.sort_values(by=['Percentage'],ascending=False)

# Машинное обучение

Закодируем категориальные признаки с помощью OneHot-кодирования:

![title](OHEnc.png)

Закодируем с помощью OneHotEncoding категориальные признаки

In [ ]:
base = pd.get_dummies(df, columns=['gender','rating','level of education','test preparation course',\
                                  'lunch'], drop_first=True, dtype = int)
base.sample()
base.info()

In [ ]:
base.head()

## Решим задачу регрессии: предскажем средний балл каждого сотрудника.

## Создадим обучающую матрицу и столбец с целевой переменной.

Создайте матрицу признаков X, состоящую только из закодированных признаков.

Создайте целевой вектор y, равный столбцу Percentage.

In [ ]:
y = df['Percentage']

X = base.copy()
X.drop(['erudition_score','communication_score','productivity','Erudition_PassStatus','OverAll_PassStatus','Percentage'], axis=1, inplace=True)

In [ ]:
X.head()

Разобъем данные на тренировочную и тестовую части.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.25)

## Будем использовать модель линейной регрессии

![title](LR.png)

Предсказание делается по формуле - сумма признаков с весами:
    
 $y = w_0 + w_1\cdot x_1 + w_2\cdot x_2 + ...$

Объявим модель и обучим её на тренировочных данных.

In [ ]:
model = LinearRegression()
model.fit(train_x,train_y)

Сделаем предсказание на тестовых данных и оценим качество.

In [ ]:
target = model.predict(test_x)

mean_squared_error(target,test_y) ** 0.5

Как интерпретировать эту ошибку?

Посмотрим глазами на предсказания.

In [ ]:
for i in range(10):
    print('answer:', test_y.values[i])
    print('predicted:', target[i])
    print()

Запишем формулу для предсказания:

$$y = 55 + 17\cdot old\_mean\_score - 3.4\cdot gender\_male + 2\cdot group\_B + 5.6\cdot group\_C + 7.2\cdot group\_D + 3.3\cdot group\_E - 4.8\cdot bachelor + 3.9\cdot high\_school - 0.7\cdot master - 3.5\cdot college - 6.3\cdot some\_high\_school + 7.7\cdot preparation\_course\_none$$

In [ ]:
model.coef_, model.intercept_, X.columns

Визуализируем коэффициенты.

In [ ]:
def visualize_coefficients(model, feature_names, n_features=6):

    coef = model.coef_.ravel()
    positive_coefficients = np.argsort(coef)[-n_features:]
    negative_coefficients = np.argsort(coef)[:n_features]
    all_coefs = np.hstack([negative_coefficients, positive_coefficients])

    plt.figure(figsize=(15, 5))
    colors = ["red" if c < 0 else "blue" for c in coef[all_coefs]]
    plt.bar(np.arange(2*n_features), coef[all_coefs], color=colors)
    feature_names = np.array(feature_names)
    plt.xticks(np.arange(1, 1+2*n_features), feature_names[all_coefs], rotation=60, ha="right")
    
visualize_coefficients(model, X.columns.values)